In [278]:
### ETAPA 1: COSECHA DE UVA ###
# Input: 290 lotes
# Output: lotes cosechados para proceso fermentación.

# Activación base de datos y librerías
import sqlite3
import random
import numpy as np
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

conn = sqlite3.connect('vinos.db')
c = conn.cursor()

In [279]:
ingresos_totales_finales = []
costos_totales_finales = []
utilidad_total_final = []
demandas_por_mercado = []

### ETAPA 1: COSECHA ###

In [280]:
for _ in range(10):
    # parametros cosecha
    periodo_ideal = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    calidades =  {'C1': [0.85, 0.95], 'C2': [0.92, 0.93], 'C3': [0.91, 0.87], 'C4': [0.95, 0.95], 'C5': [0.85, 0.85], 'C6': [0.93, 0.94]}
    umbrales = {'C1': 0.8, 'C2': 0.75, 'C3': 0.8, 'C4': 0.7, 'C5': 0.8, 'C6': 0.7}
    lotes = []
    penalizaciones = {
        'C1': [0.03, 0.02, 0.01, 0.005],
        'C2': [0.025, 0.012, 0.004, 0.002],
        'C3': [0.03, 0.02, 0.01, 0.005],
        'C4': [0.025, 0.012, 0.01, 0.002],
        'C5': [0.03, 0.02, 0.01, 0.005],
        'C6': [0.025, 0.012, 0.004, 0.002]
    }
    # Leo la base de datos y la guardo en una lista
    c.execute("SELECT * FROM lotes")
    lotes_obtenidos = c.fetchall()
    lotes.append(lotes_obtenidos)
    lotes = lotes[0]
    #Actualizo el día óptimo de cosecha según la distribución normal de cada lote en la base de datos

    # pasar lotes a lista de listas
    lotes = [list(elem) for elem in lotes]
    # Funcion que estima el dia de cosecha con media 0 y desv est 2
    def dia_estimado():
            optimal_day = np.random.normal(0, 2)
            return int(round(optimal_day))
    for lote in lotes:
        variacion_optimo = dia_estimado()
        lote[5]+=variacion_optimo
        lote.append(variacion_optimo)

    def calcular_probabilidad_lluvia(P_seco_a_lluvioso, P_lluvioso_a_lluvioso):
        # Inicializar probabilidades iniciales según el estado inicial deseado
        P_dia_seco = 1
        P_dia_lluvioso = 0
        dias = 7

        # Calcular probabilidades de estado para cada día
        for _ in range(dias):
            P_dia_seco_nuevo = (P_dia_seco * (1 - P_seco_a_lluvioso) + P_dia_lluvioso * (1 - P_lluvioso_a_lluvioso))
            P_dia_lluvioso_nuevo = (P_dia_seco * P_seco_a_lluvioso + P_dia_lluvioso * ( P_lluvioso_a_lluvioso))
            P_dia_seco, P_dia_lluvioso = P_dia_seco_nuevo, P_dia_lluvioso_nuevo

        porcentaje_lluvia = P_dia_lluvioso 
        
        return porcentaje_lluvia
    for lote in lotes:
        prob_lluvia = calcular_probabilidad_lluvia(lote[6],lote[7])
        prob_lluvia = round(prob_lluvia, 6)
        lote.append(prob_lluvia)
    # Lotes actualizados
    print(lotes)
    # Función para predecir el clima con markov
    def predecir_clima(prob_seco_lluvia, prob_lluvia_lluvia):
        clima_prediccion = []  # Lista para almacenar la predicción
        tipo_3 = []
        # Inicializar el primer día como seco
        clima_prediccion.append(0)
        tipo_3.append(0)
        # Calcular predicciones para los siguientes 18 días (13 días previos + 5 días de desviación)
        for _ in range(7):
            if clima_prediccion[-1] == 0:  # Si el día anterior fue seco
                clima_prediccion.append(1 if random.random() < prob_seco_lluvia else 0)
                tipo_3.append(0)
            else:  # Si el día anterior fue lluvioso
                clima_prediccion.append(1 if random.random() < prob_lluvia_lluvia else 0)
                if clima_prediccion[-1] == 1:
                    tipo_3.append(1 if random.random() <= 0.05 else 0)
        if sum(tipo_3) >= 1:
            perdida = True
        else:
            perdida = False
        return clima_prediccion, perdida
    def obtener_penalizaciones(tipo_uva, calidad, clima_prediccion):
        lista_penalizaciones = penalizaciones[tipo_uva]
        index = 7
        while index != 0:
            if index == 7:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[0])
            elif index == 6:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[1])
            elif index == 5:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[2])
            elif index == 4:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 3:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 2:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 1:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            index -= 1
        return calidad

    def obtener_penalizaciones_2(tipo_uva, calidad, clima_prediccion):
        lista_penalizaciones = penalizaciones[tipo_uva]
        index = 5
        while index != 0:
            if index == 5:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[0])
            elif index == 4:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[1])
            elif index == 3:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[2])
            elif index == 2:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 1:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            index -= 1
        return calidad


    # funcion que entrega el valor final de la calidad considerando la lluvia
    def calcular_calidad(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
        t_minus = -7
        t_plus = 7
        A = np.array([
            [t_minus**2, t_minus, 1],
            [0, 0, 1],
            [t_plus**2, t_plus, 1]])
        b = np.array([q_t_minus, 1, q_t_plus])
        coefficients = np.linalg.solve(A, b)
        a, b, c = coefficients
        def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
            # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
            calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
            calidad_final = obtener_penalizaciones(tipo_uva, calidad_sin_lluvia, clima_prediccion)
            return calidad_final
        q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
        return q_t_optimal

    def calcular_calidad_2(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
        t_minus = -7
        t_plus = 7
        A = np.array([
            [t_minus**2, t_minus, 1],
            [0, 0, 1],
            [t_plus**2, t_plus, 1]])
        b = np.array([q_t_minus, 1, q_t_plus])
        coefficients = np.linalg.solve(A, b)
        a, b, c = coefficients
        def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
            # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
            calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
            calidad_final = obtener_penalizaciones_2(tipo_uva, calidad_sin_lluvia, clima_prediccion)
            return calidad_final
        q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
        return q_t_optimal
    # Parametros para encontrar en la simulacion etapa 1
    calidades_lotes = []
    lotes_perdidos = []
    lotes_obtenidos = []
    litros_totales_perdidos = 0
    litros_totales_obtenidos = 0
    botellas_totales_perdidas = 0
    botellas_totales_obtenidas = 0
    costos_totales_perdidos = 0
    costos_totales_obtenidos = 0
    #Se crea ua funcion que nos entregue una lista que corresponde a la cantidad de lluvia maxima tolerada por cada una de las cepas
    #Esta funcion se puede alterar dependiendo de que % de lotes se quiere comprar con compra spot
    def tolerancia_cepa(lotes):
        cepas = ["C1","C2","C3","C4","C5","C6"]
        tolerancia = []

        penalizaciones = {'C1': [0.03, 0.02, 0.01, 0.005],
                            'C2': [0.025, 0.012, 0.004, 0.002],
                            'C3': [0.03, 0.02, 0.01, 0.005],
                            'C4': [0.025, 0.012, 0.01, 0.002],
                            'C5': [0.03, 0.02, 0.01, 0.005],
                            'C6': [0.025, 0.012, 0.004, 0.002]
                            }
        #Se hacen dos litas por cada cepa, donde una corresponde a la calidad o ponterador de calidad obtenido en base a una de las partes de la ecuacion de calidad
        #(la parte que pensaliza por la presencia de lluvias en cada uno de los dias previos)
        #Luego de eso se determina un porcentaje de los lotes que será comprado con contrato spot (Ej: el 20 porciento que contenga a los lotes con peor calidad)
        for cepa in cepas:
            valores_lluvia = []
            valores_calidad = []
            for lote in lotes:
                porcentaje_lluvia = lote[10]
                lote_cepa = lote[3]
                if cepa == lote_cepa:
                    calidad = ((1-penalizaciones[cepa][0]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][1]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][2]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia)
                        )
                    valores_lluvia.append(porcentaje_lluvia)
                    valores_calidad.append(calidad)
            
            #Se ordenan las listas de mayor a menor para luego poder obtener que valor corresponde al % deseado
            valores_lluvia.sort(reverse = True)
            valores_calidad.sort(reverse = True)

            #En este caso el 0.2 es porque buscamos la calidad en el 20% mas bajo y con su posicion encontraremos la lluvia que genera la calidad del 20% mas bajo
            posicion_porcentaje = int(len(valores_calidad) * 0.2)
            valor_lluvia = valores_lluvia[posicion_porcentaje]
            tolerancia.append(valor_lluvia)
        return tolerancia             


    #Creamos una funcion a la que se le entregue un lote y una lista con el porcentaje de lluvia tolerado por cepa y en base a eso
    #se decide que tipo de contrato se escogerá
    def tipo_contrato(tolerancia, lote):
        cepa = lote[3]
        porcentaje_lluvia = lote[10]
        if cepa == "C1":
            tolerancia_cepa = tolerancia[0]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C2":
            tolerancia_cepa = tolerancia[1]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C3":
            tolerancia_cepa = tolerancia[2]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0
            
        elif cepa == "C4":
            tolerancia_cepa = tolerancia[3]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C5":
            tolerancia_cepa = tolerancia[4]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C6":
            tolerancia_cepa = tolerancia[5]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        return contrato
    climas_spot = []
    climas_spot_3 = []
    # Simulación de la etapa 1

    tolerancia_lluvias = tolerancia_cepa(lotes)

    for t in range(50, 150):
        for lote in lotes:
            #El siguiente valor corresponde a el contrato que se utilizara en cada lote
            contrato_lote = tipo_contrato(tolerancia_lluvias,lote)
            lote_id = lote[0]
            lote_cod = lote[1]
            lote_numero = lote[2]
            lote_tipo_uva = lote[3]
            lote_toneladas = lote[4]
            lote_dia_optimo = lote[5]
            lote_prob_seco_lluvia = lote[6]
            lote_prob_lluvia_lluvia = lote[7]
            lote_precio_usd = lote[8]
            lote_variacion = lote[9]
            lote_prob_lluvia = lote[10]
            if t == lote_dia_optimo - 8:
                lote_clima_prediccion, perdida = predecir_clima(lote_prob_seco_lluvia, lote_prob_lluvia_lluvia)
                lista = lote
                q_t = calidades[lote_tipo_uva]
                lote_q_t_minus = q_t[0]
                lote_q_t_plus = q_t[1]
                calidad_lote = calcular_calidad(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion, lote_tipo_uva)
                calidades_lotes.append(calidad_lote)
                lista.append(calidad_lote)
                lista.append(contrato_lote)
                lista.append(perdida)
                lotes_obtenidos.append(lista)
                if contrato_lote > 0:
                    calidad_lote = calcular_calidad_2(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion[:-2], lote_tipo_uva)
                    if not perdida:
                        climas_spot.append(calidad_lote)
                    elif perdida:
                        climas_spot_3.append(calidad_lote)
                    
                    
                    
    print(lotes_obtenidos)
    #lote_id = lote[0]
    #lote_cod = lote[1]
    #lote_numero = lote[2]
    #lote_tipo_uva = lote[3]
    #lote_toneladas = lote[4]
    #lote_dia_optimo = lote[5]
    #ote_prob_seco_lluvia = lote[6]
    #lote_prob_lluvia_lluvia = lote[7]
    #lote_precio_usd = lote[8]
    #lote_variacion = lote[9]
    #lote_prob_lluvia = lote[10]
    #calidad lote
    #contrato lote
    #tipo 3(true or false)
    # Modificamos los lotes obtenidos y perdidos con la info relevante, es decir, 
    # sacamos las probabilidades de lluvia y el día óptimo de cosecha, entre otras cosas.
    lista_final = []
    for lote in lotes_obtenidos:
        lista = []
        lote_cod = lote[1]
        lote_numero = lote[2]
        lote_tipo_uva = lote[3]
        lote_toneladas = lote[4]
        lote_costo = lote[8]
        lote_calidad = lote[11]
        lote_contrato = lote[12]
        lote_optimo = lote[5]
        lote_fermentacion = False
        lote_tipo_3 = lote[13]
        lote_prob_lluvia = lote[10]
        lista.append(lote_cod)
        lista.append(lote_numero)
        lista.append(lote_tipo_uva)
        lista.append(lote_toneladas)
        lista.append(lote_costo)
        lista.append(lote_calidad)
        lista.append(lote_contrato)
        lista.append(lote_optimo)
        lista.append(lote_fermentacion)
        lista.append(lote_tipo_3)
        lista.append(lote_prob_lluvia)  
        lista_final.append(lista)

    print(lista_final)
    #lote_cod = lote[1]
    #lote_numero = lote[2]
    #lote_tipo_uva = lote[3]
    #lote_toneladas = lote[4]
    #lote_costo = lote[8]
    #lote_calidad = lote[11]
    #lote_contrato = lote[12]
    #lote_optimo = lote[5]
    #lote_fermentacion = False
    #lote_tipo_3 = lote[13]
    #lote_prob_lluvia = lote[10]

    #Abajo está la recapitulación de la información de la cosecha
    lotes_obtenidos = lista_final
    # Encontrar la info que queramos de esta etapa
    print(f'Calidad máxima: {max(calidades_lotes)}')
    print(f'Calidad mínima: {min(calidades_lotes)}')
    print('')
    contador = 0
    contador_3 = 0
    for lote in lista_final:
        lote_cod = lote[0]
        lote_numero = lote[1]
        lote_tipo_uva = lote[2]
        lote_toneladas = lote[3]
        lote_costo = lote[4]
        lote_calidad = lote[5]
        lote_contrato = lote[6]
        lote_optimo = lote[7]
        lote_fermentacion = lote[8]
        lote_tipo_3 = lote[9]
        lote_prob_lluvia = lote[10]
        if not lote_tipo_3:
            kilos = lote_toneladas * 1000
            litros = kilos * 0.5
            litros_totales_obtenidos += litros
            costo_lote = kilos*lote[4]
            if lote[6] == 0:
                costo_lote = costo_lote * 0.8
                costos_totales_obtenidos += costo_lote
            elif lote[6] == 1:
                calidad_spot = climas_spot[contador]
                costos_totales_obtenidos += costo_lote*calidad_spot
                contador += 1
        elif lote_tipo_3:
            kilos = lote_toneladas * 1000
            litros = kilos * 0.5
            litros_totales_perdidos += litros
            costo_lote = kilos*lote[4]
            if lote[6] == 0:
                costo_lote = costo_lote * 0.8
                costos_totales_perdidos += costo_lote
            elif lote[6] == 1:
                costo_lote = costo_lote
                calidad_spot_3 = climas_spot_3[contador_3]
                costos_totales_perdidos += costo_lote*calidad_spot_3
    botellas_totales_obtenidas = litros_totales_obtenidos / 0.75
    print(f'Litros totales obtenidos: {round(litros_totales_obtenidos)}')
    print(f'Botellas totales obtenidas: {round(botellas_totales_obtenidas)}')
    print(f'Costos totales esperadas USD (considerando contratos spot y foward): {round(costos_totales_obtenidos)}')
    print('')
    botellas_totales_perdidas = litros_totales_perdidos / 0.75
    print(f'Litros totales perdidos: {round(litros_totales_perdidos)}')
    print(f'Botellas totales perdidas: {round(botellas_totales_perdidas)}')
    print(f'Costos totales perdidos USD (compra spot): {round(costos_totales_perdidos)}')
    print('')
    Costos_totales = costos_totales_obtenidos + costos_totales_perdidos
    print(f'Costos totales etapa 1: {round(Costos_totales)}')

    ### ETAPA 2: CALIDAD DE UVA ###
    # Parámetros simulación etapa 2
    costo_recuperado = 0
    contador = 0

    for lote in lista_final:
        lote_cod = lote[0]
        lote_numero = lote[1]
        lote_tipo_uva = lote[2]
        lote_toneladas = lote[3]
        lote_costo = lote[4]
        lote_calidad = lote[5]
        lote_contrato = lote[6]
        lote_optimo = lote[7]
        lote_fermentacion = lote[8]
        lote_tipo_3 = lote[9]
        lote_prob_lluvia = lote[10]
        if lote_tipo_3:
            lote_calidad = 0


        calidad = lote[5]
        cepa = lote[2]
        precio = lote[4]
        kilos = lote[3]*1000
        umbral = umbrales[lote_tipo_uva]
        #Obtenemos costo por lote dependiendo de como ha sido comprado
        ### 1 spot, 0 foward
        if lote_contrato == 0:
            costo = precio * 0.8 * kilos
        elif lote_contrato == 1:
            costo = precio * kilos
        if 0.5 <= lote_calidad < umbral:
            recuperado = costo * 0.3 
        if lote_calidad < 0.5:
            recuperado = costo * 0.05
        else:
            contador += 1
            recuperado = 0
        costo_recuperado += recuperado
    Costos_totales -= costo_recuperado

    print(f'Costos recuperados etapa 2: {round(costo_recuperado)} USD')
    print(f'Costos totales etapa 2, post recuperación: {round(Costos_totales)}')

    ### ETAPA 3: FERMENTACIÓN ###
    # calcula la duracion en fermentación
    def calcular_duracion_vinificacion():
        duracion_aleatoria = random.uniform(7, 9)
        dia_exacto = round(duracion_aleatoria)
        return dia_exacto
    # Parámetros simulación etapa 3
    costo_plantas = {1:[1600, 1840, 1840], 2:[1500, 1725, 1725], 3:[1800, 2070, 2070]}
    uso_plantas = [[0, 0, 0],[0, 0, 0],[0, 0, 0]]
    litros_perdidos = []
    costos_perdidos = []
    dias_tanques = []

    def espacio_tanques(uso_plantas):
        espacio_disponible = 0
        for planta in uso_plantas:
            if sum(planta) < 72:
                espacio = 72 - sum(planta)
                espacio_disponible += espacio
        return espacio_disponible
    def entrar_tanques(litros, costo):
        botellas = 0
        litros_sobrantes = 0
        condicion = False
        while not condicion:
            if litros > 25000:
                litros -= 25000
                botellas += 1
            else:
                litros_sobrantes = litros
                litros_perdidos.append(litros_sobrantes)
                costo_perdido = costo * (litros_sobrantes/0.5)
                costos_perdidos.append(costo_perdido)
                condicion = True
        return botellas
    def uso_tanques(botellas):
        while botellas > 0:
            if uso_plantas[1][0] < 24:
                uso_plantas[1][0] += 1
                botellas -= 1
            elif uso_plantas[0][0] < 24:
                uso_plantas[0][0] += 1
                botellas -= 1
            elif uso_plantas[1][1] < 24:
                uso_plantas[1][1] += 1
                botellas -= 1
            elif uso_plantas[1][2] < 24:
                uso_plantas[1][2] += 1
                botellas -= 1
            elif uso_plantas[2][0] < 24:
                uso_plantas[2][0] += 1
                botellas -= 1
            elif uso_plantas[0][1] < 24:
                uso_plantas[0][1] += 1
                botellas -= 1
            elif uso_plantas[0][2] < 24:
                uso_plantas[0][2] += 1
                botellas -= 1
            elif uso_plantas[2][1] < 24:
                uso_plantas[2][1] += 1
                botellas -= 1
            elif uso_plantas[2][2] < 24:
                uso_plantas[2][2] += 1
                botellas -= 1
            else:
                break
        return botellas
    def sacar_tanques(botellas):
        while botellas > 0:
            if uso_plantas[1][0] > 0:
                uso_plantas[1][0] -= 1
                botellas -= 1
            elif uso_plantas[0][0] > 0:
                uso_plantas[0][0] -= 1
                botellas -= 1
            elif uso_plantas[1][1] > 0:
                uso_plantas[1][1] -= 1
                botellas -= 1
            elif uso_plantas[1][2] > 0:
                uso_plantas[1][2] -= 1
                botellas -= 1
            elif uso_plantas[2][0] > 0:
                uso_plantas[2][0] -= 1
                botellas -= 1
            elif uso_plantas[0][1] > 0:
                uso_plantas[0][1] -= 1
                botellas -= 1
            elif uso_plantas[0][2] > 0:
                uso_plantas[0][2] -= 1
                botellas -= 1
            elif uso_plantas[2][1] > 0:
                uso_plantas[2][1] -= 1
                botellas -= 1
            elif uso_plantas[2][2] > 0:
                uso_plantas[2][2] -= 1
                botellas -= 1
            else:
                break
    # Más parámetros
    lista_fermentacion = []
    lista_espera = []
    litros_perdidos = []
    for t in range(30, 150):
        for lote in lista_final:
            lote_cod = lote[0]
            lote_numero = lote[1]
            lote_tipo_uva = lote[2]
            lote_toneladas = lote[3]
            lote_costo = lote[4]
            lote_calidad = lote[5]
            lote_contrato = lote[6]
            lote_dia_cosecha = lote[7]
            lote_salida_fermentacion = lote[8]
            lote_tipo_3 = lote[9]
            lote_duracion = calcular_duracion_vinificacion()
            if t == lote_dia_cosecha:
                if lote_tipo_3:
                    # los lotes que tuvieron lluvias tipo 3 se pierden y no entran a fermentación
                    pass
                elif not lote_tipo_3:
                    ### tomar en consideración que cuando dice botellas me refiero a tanques de 
                    ### 25000 litros cada uno.
                    litros_uva = lote_toneladas * 1000 * 0.5
                    botellas = entrar_tanques(litros_uva, lote_costo)
                    botellas_restantes = uso_tanques(botellas)
                    botellas_fermentadas = botellas - botellas_restantes
                    lista = [t + lote_duracion, botellas]
                    dias_tanques.append(lista)
                    if botellas_restantes > 0:
                        print(f'Se perdieron {botellas_restantes} tanques el día {t}')
                        print(botellas_restantes)
                        costo_perdido = lote_costo * ((botellas_restantes*25000)/0.5)
                        print(f'Con el costo de {costo_perdido} USD')
                        print('')
                        costos_perdidos.append(costo_perdido)
                    lote[8] = lote_duracion + t
                    litros_fermentados = botellas_fermentadas * 25000
                    lista_fermentacion.append([lote_tipo_uva, litros_fermentados, t + lote_duracion])
            if lote_salida_fermentacion == t:
                for dias in dias_tanques:
                    if dias[0] == t:
                        botella = dias[1]
                        sacar_tanques(botella)

    # lista para mandar a barricas
    print(lista_fermentacion)
    costos_perdidos = round(sum(costos_perdidos))
    print(f'Costos perdidos por no entrar a tanques: {costos_perdidos}')
    Costos_totales += sum([1600, 1840, 1840, 1500, 1725, 1725, 1800, 2070, 2070])
    Costos_totales = round(Costos_totales)
    print(f'Costos totales hasta etapa 3 fermentación: {Costos_totales}')

    ### ETAPA 4: CREACIÓN DE BLENDS Y SATISFACCIÓN DE MERCADO
    ingresos_totales = 0
    import random
    from cmath import inf
    from re import X
    from xml.sax import xmlreader


    data = [
        ("Blend 1", 1, 0.1, 0.2, 0.0, 0.3, 0.0, 0.4),
        ("Blend 2", 1, 0.2, 0.0, 0.2, 0.2, 0.0, 0.4),
        ("Blend 3", 1, 0.5, 0.0, 0.2, 0.0, 0.1, 0.2),
        ("Blend 4", 1, 0.12, 0.15, 0.08, 0.1, 0.1, 0.45),
        ("C1", 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0),
        ("C2", 1, 0.0, 1, 0.0, 0.0, 0.0, 0.0),
        ("C3", 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0),
        ("C4", 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0),
        ("C5", 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0),
        ("C6", 1, 0.0, 0.0, 0.0, 0.0, 0.0, 1)
    ]

    recipe_dict = {}
    precio_mercado = {"demanda_mercado_a": 4, "demanda_mercado_b": 2.8, "demanda_mercado_c": 2.1, "demanda_mercado_d": 1.5}
    precio_cepa = {"C1": 0.27118, "C2": 0.20381, "C3": 0.297487, "C4": 0.135593, "C5": 0.405931, "C6": 0.176233, "Blend 1":0.179, "Blend 2": 0.21134, "Blend 3": 0.27, "Blend 4": 0.22}
    costos_acumulados = {"C1": 0, "C2": 0, "C3": 0, "C4": 0, "C5": 0, "C6": 0}
    nombre_cepa = ["C1","C2","C3","C4","C5","C6"]
    relaciones_mercado_cepa = {
        'demanda_mercado_a': ['C5'],
        'demanda_mercado_b': ['C1', 'C3', 'Blend 2', 'Blend 3'],
        'demanda_mercado_c': ['C2', 'Blend 4'],
        'demanda_mercado_d': ['C4', 'C6', 'Blend 1']
    }      


    ingreso = {"demanda_mercado_a": 0, "demanda_mercado_b": 0, "demanda_mercado_c": 0, "demanda_mercado_d": 0}
    cepa_inicio = {"C1": 0,"C2": 0,"C3": 0,"C4": 0,"C5": 0, "C6": 0, "Blend 1": 0, "Blend 2": 0, "Blend 3": 0, "Blend 4": 0}
    for row in data:
        blend = row[0]
        receta = row[1]
        c_values = row[2:]
        if blend not in recipe_dict:
            recipe_dict[blend] = {}
        recipe_dict[blend][receta] = c_values


    demanda_mercado = [
        ["demanda_mercado_a", 9520000],
        ["demanda_mercado_b", 12693333],
        ["demanda_mercado_c", 11106667],
        ["demanda_mercado_d", 9520000]
    ]


    def procesar_blend(blend):
        c_values = recipe_dict[blend][receta] 
        c_values = list(c_values)
        # Obtener las proporciones de la receta para la mezcla
        # Calcular los factores de escala para cada cepa
        scale_factors = [litro_cepa[i] / (c_values[i]) if c_values[i] != 0 else float('inf') for i in range(0,len(c_values))]
        # Encontrar el factor de escala mínimo
        min_scale_factor = min(scale_factors)
        return min_scale_factor


    def ordenar_demanda(demanda_mercado):
        # Utiliza la función sorted para ordenar la lista en orden descendente
        demanda_ordenada = sorted(demanda_mercado, key=lambda x: x[1], reverse=True)
        return demanda_ordenada


    def calcular_utilizacion_cepas(recetas):
        # Suponiendo que receta contiene los porcentajes desde C1 hasta C6
        utilizacion_cepas = {}
        # Calcular la utilización de cada cepa
        for i in range(6):
            cepa = f"C{i + 1}"  # Generar el nombre de la cepa (C1, C2, ..., C6)
            x = recetas[i]
            utilizacion_cepa = cepas_obtenidas[cepa] * x
            utilizacion_cepas[cepa] = utilizacion_cepa
        
        return utilizacion_cepas


    def suma_lista(lista):
        return sum(lista)


    def actualizar_blends():
        cant_blend_1 = procesar_blend("Blend 1")
        cant_blend_2 = procesar_blend("Blend 2")
        cant_blend_3 = procesar_blend("Blend 3")
        cant_blend_4 = procesar_blend("Blend 4")
        cepas_obtenidas["Blend 1"] = cant_blend_1
        cepas_obtenidas["Blend 2"] = cant_blend_2
        cepas_obtenidas["Blend 3"] = cant_blend_3
        cepas_obtenidas["Blend 4"] = cant_blend_4


    def actualizar_min_botellas_por_mercado():
        global min_botellas_por_mercado
        min_botellas_por_mercado = {
        "Blend 1": demanda_mercado[3][1],  # Asociar Blend 1 con Mercado D, si es necesario
        "Blend 2": demanda_mercado[1][1],  # Asociar Blend 2 con Mercado B, si es necesario
        "Blend 3": demanda_mercado[1][1],  # Asociar Blend 3 con Mercado B, si es necesario
        "Blend 4": demanda_mercado[2][1],  # Asociar Blend 4 con Mercado C, si es necesario
        "C1": demanda_mercado[1][1],       # Asociar C1 con Mercado B, si es necesario
        "C2": demanda_mercado[2][1],       # Asociar C2 con Mercado C, si es necesario
        "C3": demanda_mercado[1][1],       # Asociar C3 con Mercado B, si es necesario
        "C4": demanda_mercado[3][1],       # Asociar C4 con Mercado D, si es necesario
        "C5": demanda_mercado[0][1],       # Asociar C5 con Mercado A, si es necesario
        "C6": demanda_mercado[3][1],       # Asociar C6 con Mercado D, si es necesario
        }      


    def cepas_relacionadas(mercado):
        actualizar_min_botellas_por_mercado()
        return relaciones_mercado_cepa[mercado]


    def cantidades_cepa(cepas):
        cantidad = []
        for cepa in cepas:
            x = cepas_obtenidas[cepa]
            cantidad.append(x)
        return cantidad


    def sumar_valores_cepas_ce(lista_blend_cepas, lista_valores):
        suma_cepas_c = 0
        for i in range(len(lista_blend_cepas)):
            elemento = lista_blend_cepas[i]
            print(elemento)
            if elemento.startswith('C'):
                suma_cepas_c += lista_valores[i]
        return suma_cepas_c


    def mover_cepas_al_principio(lista_blend_cepas, lista_valores):
        contador = 0
        cepas = []
        otros_elementos = []
        valores_cepas = []
        valores_otros = []

        for i, elemento in enumerate(lista_blend_cepas):
            if elemento.startswith('C'):
                cepas.append(elemento)
                valores_cepas.append(lista_valores[i])
                if lista_valores[i] != 0:
                    contador += 1
            else:
                otros_elementos.append(elemento)
                valores_otros.append(lista_valores[i])


        lista_final = list(cepas) + otros_elementos
        valores_final = list(valores_cepas) + valores_otros

        return lista_final, valores_final, contador


    def encontrar_blend_mayor_cantidad(lista_blend_cepas, lista_valores):
        max_valor = 0
        blend_mayor_cantidad = ""
        for i in range(len(lista_blend_cepas)):
            elemento = lista_blend_cepas[i]
            if elemento.startswith('Blend') and lista_valores[i] > max_valor:
                max_valor = lista_valores[i]
                blend_mayor_cantidad = elemento
        
        return blend_mayor_cantidad, max_valor


    def enviar_sobras_a_mercado(c1, c2, c3, c4, c5, c6, litro_cepa, nombre):
        # Asociamos las cepas a sus mercados correspondientes
        mercado_a = [("C5", c5)]
        mercado_b = [("C1", c1), ("C3", c3)]
        mercado_c = [("C2", c2)]
        mercado_d = [("C4", c4), ("C6", c6)]
        litro_cepa[0] -= cepas_obtenidas["C1"]
        litro_cepa[1] -= cepas_obtenidas["C2"]
        litro_cepa[2] -= cepas_obtenidas["C3"]
        litro_cepa[3] -= cepas_obtenidas["C4"]
        litro_cepa[4] -= cepas_obtenidas["C5"]
        litro_cepa[5] -= cepas_obtenidas["C6"]
        cepa_inicio["C1"] += cepas_obtenidas["C1"]
        cepa_inicio["C2"] += cepas_obtenidas["C2"]
        cepa_inicio["C3"] += cepas_obtenidas["C3"]
        cepa_inicio["C4"] += cepas_obtenidas["C4"]
        cepa_inicio["C5"] += cepas_obtenidas["C5"]
        cepa_inicio["C6"] += cepas_obtenidas["C6"]

        for mercado, cepas in [("mercado_a", mercado_a), ("mercado_b", mercado_b), ("mercado_c", mercado_c), ("mercado_d", mercado_d)]:
            total_enviado = 0
            mensaje = f"El {mercado.replace('_', ' ')} recibió"
            cepas_enviadas = []
            litro_cepa = [0,0,0,0,0,0]

            for cepa, cantidad in cepas:
                # Distribuir las cantidades a los mercados correspondientes
                cepas_obtenidas[cepa] -= cepas_obtenidas[cepa]
                costo_cepa = cepas_obtenidas[cepa] * precio_cepa[cepa]
                
                for demanda_mercado_actual in demanda_mercado:
                    if demanda_mercado_actual[0] == f"demanda_{mercado}":
                        demanda_mercado_actual[1] -= cantidad

            if total_enviado > 0:
                mensaje += " " + " y ".join([f"{cantidad} litros de {cepa}" for cepa, cantidad in cepas_enviadas]) 
                print(mensaje)
        actualizar_blends()
                
        print(demanda_mercado)


    def procesar_mercado(mercado, nombre_mercado_demanda):
        for i, mercado_item in enumerate(mercado):
            if mercado_item[0] == nombre_mercado_demanda:
                mercado[i][1] -= lista_valores[0]



    def sumar_blend(blend_mayor):
        cepa_inicio[blend_mayor] += cepa_inicio[blend_mayor]


    def procesar_receta(primer_nombre, recetas, tipo, nombre):
        utilizacion_cepas = calcular_utilizacion_cepas(recetas)
        print(utilizacion_cepas,"UUU")
        if tipo == "cepa":
            if nombre == "C1":
                litro_cepa[0] -= utilizacion_cepas["C1"]
                cepa_inicio["C1"] += cepas_obtenidas["C1"]

            
            elif nombre == "C2":
                litro_cepa[1] -= utilizacion_cepas["C2"]
                cepa_inicio["C2"] += cepas_obtenidas["C2"]

            elif nombre == "C3":
                litro_cepa[2] -= utilizacion_cepas["C3"]
                cepa_inicio["C3"] += cepas_obtenidas["C3"]

            elif nombre == "C4":
                litro_cepa[3] -= utilizacion_cepas["C4"]
                cepa_inicio["C4"] += cepas_obtenidas["C4"]


            elif nombre == "C5":
                litro_cepa[4] -= utilizacion_cepas["C5"]
                cepa_inicio["C5"] += cepas_obtenidas["C5"]


            elif nombre == "C6":
                litro_cepa[5] -= utilizacion_cepas["C6"]
                cepa_inicio["C6"] += cepas_obtenidas["C6"]


        elif tipo == "Blend":
            if nombre == "Blend 1":
                cepa_inicio["Blend 1"] += cepas_obtenidas["Blend 1"]
                cepas_obtenidas["Blend 1"] -= cepas_obtenidas["Blend 1"]
            elif nombre == "Blend 2":
                cepa_inicio["Blend 2"] += cepas_obtenidas["Blend 2"]
                cepas_obtenidas["Blend 2"] -= cepas_obtenidas["Blend 2"]
            elif nombre == "Blend 3":
                cepa_inicio["Blend 3"] += cepas_obtenidas["Blend 3"]
                cepas_obtenidas["Blend 3"] -= cepas_obtenidas["Blend 3"]
            elif nombre == "Blend 4":
                cepa_inicio["Blend 4"] += cepas_obtenidas["Blend 4"]
                cepas_obtenidas["Blend 4"] -= cepas_obtenidas["Blend 4"]
            litro_cepa[0] -= utilizacion_cepas["C1"]
            litro_cepa[1] -= utilizacion_cepas["C2"]
            litro_cepa[2] -= utilizacion_cepas["C3"]
            litro_cepa[3] -= utilizacion_cepas["C4"]
            litro_cepa[4] -= utilizacion_cepas["C5"]
            litro_cepa[5] -= utilizacion_cepas["C6"]
        
        for cepa in nombre_cepa:
            cepas_obtenidas[cepa] -= utilizacion_cepas[cepa]
            costo_cepa = utilizacion_cepas[cepa] * precio_cepa[cepa]





    # Restricciones mínimas por mercado (ajustadas)
    min_botellas_por_mercado = {
        "Blend 1": demanda_mercado[3][1],  # Asociar Blend 1 con Mercado D, si es necesario
        "Blend 2": demanda_mercado[1][1],  # Asociar Blend 2 con Mercado B, si es necesario
        "Blend 3": demanda_mercado[1][1],  # Asociar Blend 3 con Mercado B, si es necesario
        "Blend 4": demanda_mercado[2][1],  # Asociar Blend 4 con Mercado C, si es necesario
        "C1": demanda_mercado[1][1],       # Asociar C1 con Mercado B, si es necesario
        "C2": demanda_mercado[2][1],       # Asociar C2 con Mercado C, si es necesario
        "C3": demanda_mercado[1][1],       # Asociar C3 con Mercado B, si es necesario
        "C4": demanda_mercado[3][1],       # Asociar C4 con Mercado D, si es necesario
        "C5": demanda_mercado[0][1],       # Asociar C5 con Mercado A, si es necesario
        "C6": demanda_mercado[3][1],       # Asociar C6 con Mercado D, si es necesario
    }


    def suma_demanda():
        suma = 0
        for i in demanda_mercado:
            suma += i[1]
        return suma


    # Inicializamos las variables para los litros en cada mercado




    # Inicializamos las variables para los litros sobrantes por mercado
    sobrantes_a = 0
    sobrantes_b = 0
    sobrantes_c = 0
    sobrantes_d = 0

    cepas_obtenidas = {
            'C1': 0,
            'C2': 0,
            'C3': 0,
            'C4': 0,
            'C5': 0,
            'C6': 0,
        }

    litro_cepa = [0 , 0, 0, 0, 0, 0]
    contador_dias_blend4 = 0
    for t in range(65, 150):
        cant = 0
        cepa_c1 = 0
        cepa_c2 = 0
        cepa_c3 = 0
        cepa_c4 = 0
        cepa_c5 = 0
        cepa_c6 = 0
        print(f"------{t}--------------------")
        for lote in lista_fermentacion:
            if lote[2] == t:
                if lote[0] == "C1":
                    cepa_c1 += lote[1] 
                    cant += 1
        
                if lote[0] == "C2":
                    cepa_c2 += lote[1]
                    cant += 1
                
                if lote[0] == "C3":
                    cepa_c3 += lote[1]
                    cant += 1
                
                if lote[0] == "C4":
                    cepa_c4 += lote[1]
                    cant += 1 
                
                if lote[0] == "C5":
                    cepa_c5 += lote[1]
                    cant += 1
                
                if lote[0] == "C6":
                    cepa_c6 += lote[1] 
                    cant += 1   
        if cant == 0:
            print("NO HAY CEPAS ESTE DIA")
            pass
        else:
            print(cepa_c1, cepa_c2, cepa_c3, cepa_c4, cepa_c5, cepa_c6,"CEPAS")
            cepas_obtenidas['C1'] += cepa_c1
            cepas_obtenidas['C2'] += cepa_c2    
            cepas_obtenidas['C3'] += cepa_c3
            cepas_obtenidas['C4'] += cepa_c4
            cepas_obtenidas['C5'] += cepa_c5
            cepas_obtenidas['C6'] += cepa_c6
            litro_cepa[0] += cepa_c1
            litro_cepa[1] += cepa_c2
            litro_cepa[2] += cepa_c3
            litro_cepa[3] += cepa_c4
            litro_cepa[4] += cepa_c5
            litro_cepa[5] += cepa_c6
            cant_blend_1 = procesar_blend("Blend 1")
            cant_blend_2 = procesar_blend("Blend 2")
            cant_blend_3 = procesar_blend("Blend 3")
            cant_blend_4 = procesar_blend("Blend 4")
            cepas_obtenidas["Blend 1"] = cant_blend_1
            cepas_obtenidas["Blend 2"] = cant_blend_2
            cepas_obtenidas["Blend 3"] = cant_blend_3
            cepas_obtenidas["Blend 4"] = cant_blend_4
            print(cepas_obtenidas,"inicio")
            x = suma_demanda()
            if x > 0:
            # Inicializamos las variables para las cepas utilizadas por día
                demanda_mercado_ordenada = ordenar_demanda(demanda_mercado)
                mercado_mayor_demanda = demanda_mercado_ordenada[0][1]
                nombre_mercado_mayor_demanda = demanda_mercado_ordenada[0][0]
                    # Accede a las cepas relacionadas a ese mercado desde el diccionario min_botellas_por_mercado
            else:
                pos = random.randint(0, 3)
                mercado_mayor_demanda = demanda_mercado_ordenada[pos][1]
                nombre_mercado_mayor_demanda = demanda_mercado_ordenada[pos][0]
            print(mercado_mayor_demanda,"demanda inicio dia", nombre_mercado_mayor_demanda)
            cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
            lista_valores = cantidades_cepa(cepas)
            print(cepas)
            print(lista_valores)
            
            suma = suma_lista(lista_valores)
            if nombre_mercado_mayor_demanda == "demanda_mercado_c" and suma != 0:
                print("HOla")
                lista_combinada = list(zip(cepas, lista_valores))
                lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                nombres_ordenados, lista_valores = zip(*lista_ordenada)
                nombre_ordenados = list(nombres_ordenados)
                lista_valores = list(lista_valores)
                x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                if max_valor < x:
                    print("h")
                    print("HOOOOOOOOO")
                    contador_dias_blend4 += 1
                    if contador_dias_blend4 == 9:

                        print("SAAAAAAAAAAAAABBBBBBBBEEEEEEEEEEEE")
                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'],litro_cepa, nombre_mercado_mayor_demanda)
                    pass
                else:
                    if max_valor >= x:
                        print("BLEEEEEEEEES")
                        procesar_mercado(demanda_mercado, nombre_mercado_mayor_demanda)
                        primer_nombre = blend_mayor
                        print(primer_nombre)  # Obtener el primer nombre de la lista
                        # Buscar las recetas correspondientes en data
                        recetas = None
                        for item in data:
                            if item[0] == primer_nombre:
                                recetas = item[2:]  # Excluir el primer elemento (nombre)
                                break
                        print(recetas)
                        print(lista_valores)
                        procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", primer_nombre)
                        print(lista_valores[0])
                        litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                        actualizar_blends()
                        print(cepas_obtenidas, "cepas_obtenidas")
                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                        print("1")

                    
                            
            else:
                if suma == 0: #entonces pasar al siguiente mercado dado que no existe cepas para producir en este mercado
                    mercado_mayor_demanda = demanda_mercado_ordenada[1][1]
                    print(mercado_mayor_demanda)
                    nombre_mercado_mayor_demanda = demanda_mercado_ordenada[1][0]
                    print(nombre_mercado_mayor_demanda)
                    cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
                    print(cepas)
                    lista_valores = cantidades_cepa(cepas)
                    print(lista_valores)
                    suma = suma_lista(lista_valores)
                    if nombre_mercado_mayor_demanda == "demanda_mercado_c" and suma != 0:
                        print("HOla")
                        lista_combinada = list(zip(cepas, lista_valores))
                        lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                        nombres_ordenados, lista_valores = zip(*lista_ordenada)
                        nombre_ordenados = list(nombres_ordenados)
                        lista_valores = list(lista_valores)
                        x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                        blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                        if max_valor < x:
                            print("h")
                            print("HOOOOOOOOO")
                            contador_dias_blend4 += 1
                            if contador_dias_blend4 == 9:

                                print("SAAAAAAAAAAAAABBBBBBBBEEEEEEEEEEEE")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'],litro_cepa, nombre_mercado_mayor_demanda)
                            pass
                        else:
                            if max_valor >= x:
                                print("BLEESSSSSS")
                                procesar_mercado(demanda_mercado, nombre_mercado_mayor_demanda)
                                primer_nombre = blend_mayor
                                print(primer_nombre)  # Obtener el primer nombre de la lista
                                # Buscar las recetas correspondientes en data
                                recetas = None
                                for item in data:
                                    if item[0] == primer_nombre:
                                        recetas = item[2:]  # Excluir el primer elemento (nombre)
                                        break
                                procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", primer_nombre)
                                print(lista_valores[0])
                                litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                actualizar_blends()
                                print(cepas_obtenidas, "cepas_obtenidas")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                                print("1")



                    if suma == 0:
                        mercado_mayor_demanda = demanda_mercado_ordenada[2][1]
                        print(mercado_mayor_demanda)
                        nombre_mercado_mayor_demanda = demanda_mercado_ordenada[2][0]
                        print(nombre_mercado_mayor_demanda)
                        cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
                        print(cepas)
                        lista_valores = cantidades_cepa(cepas)
                        print(lista_valores)
                        suma = suma_lista(lista_valores)
                        if suma == 0:
                            mercado_mayor_demanda = demanda_mercado_ordenada[3][1]
                            print(mercado_mayor_demanda)
                            nombre_mercado_mayor_demanda = demanda_mercado_ordenada[3][0]
                            print(nombre_mercado_mayor_demanda)
                            cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
                            print(cepas)
                            lista_valores = cantidades_cepa(cepas)
                            print(lista_valores)
                            lista_combinada = list(zip(cepas, lista_valores))
                            lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                            nombres_ordenados, lista_valores = zip(*lista_ordenada)
                            nombre_ordenados = list(nombres_ordenados)
                            lista_valores = list(lista_valores)
                            x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                            blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                            if max_valor >= x:
                                procesar_mercado(demanda_mercado, nombre_mercado_mayor_demanda)
                                primer_nombre = nombres_ordenados[0]
                                print(primer_nombre)  # Obtener el primer nombre de la lista
                                # Buscar las recetas correspondientes en data
                                recetas = None
                                for item in data:
                                    if item[0] == primer_nombre:
                                        recetas = item[2:]  # Excluir el primer elemento (nombre)
                                        break
                                procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", primer_nombre)
                                print(lista_valores[0])
                                litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                actualizar_blends()
                                print(cepas_obtenidas, "cepas_obtenidas")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                                print("1")
                            else:
                                lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                                print(lista_valores, lista_blend_cepas)
                                for i in range(0, contador):
                                    for j, mercado in enumerate(demanda_mercado):
                                        if mercado[0] == nombre_mercado_mayor_demanda:
                                            demanda_mercado[j][1] -= lista_valores[i]  
                                    primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                                    # Buscar las recetas correspondientes en data
                                    recetas = None
                                    for item in data:
                                        if item[0] == primer_nombre:
                                            recetas = item[2:]  # Excluir el primer elemento (nombre)
                                            break
                                    print(lista_valores[i])
                                    procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i])
                                    litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                    actualizar_blends()
                                    print(cepas_obtenidas, "cepas_obtenidas")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                                print("4")
                            
                        else:
                            print("E")
                            lista_combinada = list(zip(cepas, lista_valores))
                            lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                            nombres_ordenados, lista_valores = zip(*lista_ordenada)
                            nombre_ordenados = list(nombres_ordenados)
                            lista_valores = list(lista_valores)
                            x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                            blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                            if max_valor >= x:
                                procesar_mercado(demanda_mercado, nombre_mercado_mayor_demanda)
                                primer_nombre = nombres_ordenados[0]
                                print(primer_nombre)  # Obtener el primer nombre de la lista
                                # Buscar las recetas correspondientes en data
                                recetas = None
                                for item in data:
                                    if item[0] == primer_nombre:
                                        recetas = item[2:]  # Excluir el primer elemento (nombre)
                                        break
                                procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", blend_mayor)
                                print(lista_valores[0])
                                litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                actualizar_blends()
                                print(cepas_obtenidas, "cepas_obtenidas")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                                print("1")
                            else:
                                lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                                print(lista_valores, lista_blend_cepas)
                                for i in range(0, contador):
                                    for j, mercado in enumerate(demanda_mercado):
                                        if mercado[0] == nombre_mercado_mayor_demanda:
                                            demanda_mercado[j][1] -= lista_valores[i]  
                                    primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                                    # Buscar las recetas correspondientes en data
                                    recetas = None
                                    for item in data:
                                        if item[0] == primer_nombre:
                                            recetas = item[2:]  # Excluir el primer elemento (nombre)
                                            break
                                    print(lista_valores[i])
                                    procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i])
                                    litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                    actualizar_blends()
                                    print(cepas_obtenidas, "cepas_obtenidas")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                                print("4")

                    else:
                        print("w")
                        lista_combinada = list(zip(cepas, lista_valores))
                        lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                        nombres_ordenados, lista_valores = zip(*lista_ordenada)
                        nombre_ordenados = list(nombres_ordenados)
                        lista_valores = list(lista_valores)
                        x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                        blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                        if max_valor >= x:
                            lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                            nombres_ordenados, lista_valores = zip(*lista_ordenada)
                            procesar_mercado(demanda_mercado, nombre_mercado_mayor_demanda)
                            primer_nombre = nombres_ordenados[0]  # Obtener el primer nombre de la lista
                            # Buscar las recetas correspondientes en data
                            recetas = None
                            for item in data:
                                if item[0] == primer_nombre:
                                    recetas = item[2:]  # Excluir el primer elemento (nombre)
                                    break
                            print(lista_valores[0])
                            procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", blend_mayor)
                            litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                            actualizar_blends()
                            print(cepas_obtenidas, "cepas_obtenidas")
                            enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                            print("5")

                        else:
                            lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                            for i in range(0, contador):
                                for j, mercado in enumerate(demanda_mercado):
                                    if mercado[0] == nombre_mercado_mayor_demanda:
                                        demanda_mercado[j][1] -= lista_valores[i]  
                                primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                                # Buscar las recetas correspondientes en data
                                recetas = None
                                for item in data:
                                    if item[0] == primer_nombre:
                                        recetas = item[2:]  # Excluir el primer elemento (nombre)
                                        break
                                print(lista_valores[i])
                                procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i])
                                litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                actualizar_blends()
                                print(cepas_obtenidas, "cepas_obtenidas")
                            enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                            print("6")

                else:
                    lista_combinada = list(zip(cepas, lista_valores))
                    lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                    nombres_ordenados, lista_valores = zip(*lista_ordenada)
                    nombre_ordenados = list(nombres_ordenados)
                    lista_valores = list(lista_valores)
                    x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                    blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                    if max_valor >= x:
                        lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                        nombres_ordenados, lista_valores = zip(*lista_ordenada)
                        procesar_mercado(demanda_mercado, nombre_mercado_mayor_demanda)
                        primer_nombre = nombres_ordenados[0]  # Obtener el primer nombre de la lista
                        # Buscar las recetas correspondientes en data
                        recetas = None
                        for item in data:
                            if item[0] == primer_nombre:
                                recetas = item[2:]  # Excluir el primer elemento (nombre)
                                break
                        print(lista_valores[0])
                        procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", blend_mayor)
                        litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                        actualizar_blends()
                        print(cepas_obtenidas, "cepas_obtenidas")
                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                    else:
                        lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                        for i in range(0, contador):
                            for j, mercado in enumerate(demanda_mercado):
                                if mercado[0] == nombre_mercado_mayor_demanda:
                                    demanda_mercado[j][1] -= lista_valores[i]  
                            primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                            # Buscar las recetas correspondientes en data
                            recetas = None
                            for item in data:
                                if item[0] == primer_nombre:
                                    recetas = item[2:]  # Excluir el primer elemento (nombre)
                                    break
                            print(lista_valores[i])
                            procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i])
                            litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                            actualizar_blends()
                            print(cepas_obtenidas, "cepas_obtenidas")
                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda)
                        

            print(cepa_inicio, "cepa_producida")
            # Ordenar la lista combinada por los valores en orden descendente
            # Desempacar las listas ordenadas
            

        

        # Verifica si se encontró el mercado y, si es así, imprime su demanda
    costos_mercado = {"demanda_mercado_a": 0, "demanda_mercado_b": 0, "demanda_mercado_c": 0, "demanda_mercado_d": 0}

    mercado_final = {"demanda_mercado_a": 0, "demanda_mercado_b": 0, "demanda_mercado_c": 0, "demanda_mercado_d":0}
    print(demanda_mercado, "Demanda_mercado")
    # Calcular costos y sumarlos a los diccionarios correspondientes
    costos_acumulados
    x = ingreso["demanda_mercado_a"] = cepa_inicio["C5"]/0.75*precio_mercado["demanda_mercado_a"]
    y = ingreso["demanda_mercado_b"] = cepa_inicio["C1"]/0.75*precio_mercado["demanda_mercado_b"] + cepa_inicio["C3"]/0.75*precio_mercado["demanda_mercado_b"] + cepa_inicio["Blend 2"]/0.75*precio_mercado["demanda_mercado_b"] + cepa_inicio["Blend 3"]/0.75*precio_mercado["demanda_mercado_b"]
    z =ingreso["demanda_mercado_c"] = cepa_inicio["C2"]/0.75*precio_mercado["demanda_mercado_c"] + cepa_inicio["Blend 4"]/0.75*precio_mercado["demanda_mercado_c"]
    w = ingreso["demanda_mercado_d"] = cepa_inicio["C4"]/0.75*precio_mercado["demanda_mercado_d"] + cepa_inicio["C6"]/0.75*precio_mercado["demanda_mercado_d"] + cepa_inicio["Blend 1"]/0.75*precio_mercado["demanda_mercado_d"]
    print(ingreso, "INGRESOS ; SIN COSTOS RESTADOS")

    ca = costos_mercado["demanda_mercado_a"] = (cepa_inicio["C5"]/0.75)*2.5 
    cb = costos_mercado["demanda_mercado_b"] = (cepa_inicio["C1"]/0.75)*2 + (cepa_inicio["C3"]/0.75)*2 + (cepa_inicio["Blend 2"]/0.75)*2  + (cepa_inicio["Blend 3"]/0.75)*2
    cc = costos_mercado["demanda_mercado_c"] = (cepa_inicio["C2"]/0.75)*1.45 + (cepa_inicio["Blend 4"]/0.75)*1.45
    cd = costos_mercado["demanda_mercado_d"] = (cepa_inicio["C4"]/0.75)*1.17 + (cepa_inicio["C6"]/0.75)*1.17 + (cepa_inicio["Blend 1"]/0.75)*1.17


    mercado_final["demanda_mercado_a"] = cepa_inicio["C5"]
    mercado_final["demanda_mercado_b"] = cepa_inicio["C1"] + cepa_inicio["C3"] + cepa_inicio["Blend 2"] + cepa_inicio["Blend 3"]
    mercado_final["demanda_mercado_c"] = cepa_inicio["C2"] + cepa_inicio["Blend 4"]
    mercado_final["demanda_mercado_d"] = cepa_inicio["C4"] + cepa_inicio["C6"] + cepa_inicio["Blend 1"]
    lista_mercados = [mercado_final["demanda_mercado_a"], mercado_final["demanda_mercado_b"], mercado_final["demanda_mercado_c"], mercado_final["demanda_mercado_d"]]
    demandas_por_mercado.append(lista_mercados)
    print("CANTIDAD PRODUCIDA POR CEPA Y BLENDS")
    print(cepa_inicio,"producido")
    print("DEMANDA QUE SE LOGRO LLENAR POR MERCADO")
    print(mercado_final,"mercado producido")
    print("INGRESOS POR MERCADO (precio minimo)")
    print(ingreso,"Ing")
    print("COSTOS POR MERCADO (solo costo de cepa)")
    print(costos_mercado, "costo_mercado")
    COSTO_TOTAL = costos_mercado["demanda_mercado_a"] + costos_mercado["demanda_mercado_b"] + costos_mercado["demanda_mercado_c"] + costos_mercado["demanda_mercado_d"]
    print("COSTO TOTAL:", COSTO_TOTAL)
    INGRESO_TOTAL = ingreso["demanda_mercado_a"] + ingreso["demanda_mercado_b"] +  ingreso["demanda_mercado_c"] + ingreso["demanda_mercado_d"]
    print("INGRESO TOTAL:", INGRESO_TOTAL)
    Costos_totales += COSTO_TOTAL
    ingresos_totales += INGRESO_TOTAL
    Costos_totales = round(Costos_totales)
    ingresos_totales = round(ingresos_totales)
    print(Costos_totales, "Costos_totales")
    print(ingresos_totales, "ingreso_total")
    utilidad = round(ingresos_totales - Costos_totales)
    print(utilidad, "utilidad")
    ingresos_totales_finales.append(ingresos_totales)
    costos_totales_finales.append(Costos_totales)
    utilidad_total_final.append(utilidad)

[[1, 'L_1_C4', 1, 'C4', 217.5, 119, 0.43, 0.39, 0.257, 1, 0.413462], [2, 'L_2_C4', 2, 'C4', 517.5, 125, 0.35, 0.44, 0.269, 2, 0.384615], [3, 'L_3_C6', 3, 'C6', 450.0, 79, 0.46, 0.38, 0.362, 0, 0.425926], [4, 'L_4_C4', 4, 'C4', 168.75, 119, 0.32, 0.51, 0.276, -3, 0.395058], [5, 'L_5_C6', 5, 'C6', 243.75, 105, 0.49, 0.6, 0.348, 1, 0.550562], [6, 'L_6_C3', 6, 'C3', 150.0, 97, 0.31, 0.56, 0.579, 1, 0.413308], [7, 'L_7_C5', 7, 'C5', 558.75, 110, 0.43, 0.39, 0.795, 1, 0.413462], [8, 'L_8_C6', 8, 'C6', 442.5, 77, 0.38, 0.35, 0.36, -1, 0.368932], [9, 'L_9_C1', 9, 'C1', 300.0, 130, 0.36, 0.35, 0.528, 5, 0.356436], [10, 'L_10_C5', 10, 'C5', 108.75, 80, 0.5, 0.56, 0.806, 1, 0.531915], [11, 'L_11_C1', 11, 'C1', 555.0, 103, 0.31, 0.53, 0.557, 3, 0.397426], [12, 'L_12_C1', 12, 'C1', 412.5, 122, 0.41, 0.33, 0.516, -2, 0.37963], [13, 'L_13_C5', 13, 'C5', 547.5, 87, 0.5, 0.48, 0.811, -2, 0.490196], [14, 'L_14_C2', 14, 'C2', 498.75, 115, 0.44, 0.52, 0.416, -1, 0.478261], [15, 'L_15_C1', 15, 'C1', 101.25

In [281]:
def calcular_promedio(lista):
    return sum(lista) / len(lista)



In [282]:
print(ingresos_totales_finales)
print(costos_totales_finales)
print(utilidad_total_final)
print(calcular_promedio(ingresos_totales_finales))
print(calcular_promedio(costos_totales_finales))
print(calcular_promedio(utilidad_total_final))
print(demandas_por_mercado)

[148841522, 142431078, 150019178, 146010167, 152815222, 153645678, 152243844, 144953478, 147462200, 151513333]
[144592809, 140460768, 145315845, 142530930, 147456324, 147829476, 146801831, 141887934, 143567240, 146503253]
[4248713, 1970310, 4703333, 3479237, 5358898, 5816202, 5442013, 3065544, 3894960, 5010080]
148993570.0
144694641.0
4298929.0
[[9147500.0, 13372000.0, 10513472.222222222, 10347500.0], [8340000.0, 13372000.0, 9440694.444444444, 10797500.0], [9522500.0, 13101000.0, 10374861.111111112, 10636250.0], [9160000.0, 12442500.0, 10707083.333333332, 10362500.0], [9345000.0, 13565000.0, 10883055.555555556, 10930000.0], [9615000.0, 13336000.0, 11108194.444444444, 10737500.0], [9497500.0, 13086000.0, 11342361.111111112, 10488750.0], [8972500.0, 13015500.0, 9915694.444444444, 10372500.0], [9172500.0, 13448000.0, 9774583.333333334, 10483750.0], [9470000.0, 13385000.0, 10032500.0, 11472500.0]]
